In [1]:
import warnings
warnings.filterwarnings("ignore")
from src.dataset import load_tokenized_data
from src.model import LyricsMelodyCNNModel
from src.embeddings import extract_embedding_weights
from tensorflow.python.keras.callbacks import TensorBoard

Using TensorFlow backend.


In [ ]:
x, y, tokenizer, songs = ds.load_tokenized_data(with_melody=True, melody_type='CNN', max_samples=100000)
songs = songs.reshape(-1, 128,128,1)
embedding_matrix = extract_embedding_weights()

In [5]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

In [8]:
cnn_model = LyricsMelodyCNNModel(tokenizer, embedding_matrix, 
                                 rnn_type='GRU', 
                                 rnn_units=20, 
                                 bidirectional=False,
                                 dropout=0.4,
                                 train_embedding=True, 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 20) 200         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 20) 80          conv2d_1[0][0]                   
__________________________________________________________________________________________________
input_1 (InputLayer) 

In [9]:
cnn_model.train([x,songs], y, epochs=1)

Instructions for updating:
Use tf.cast instead.
Train on 90000 samples, validate on 10000 samples
Epoch 1/1
90000/90000 [==============================] - 96s 1ms/step - loss: 7.1104 - _perplexity: 208.4033 - val_loss: 6.6095 - val__perplexity: 194.3302


In [ ]:
### LOAD Test Data

In [14]:
# cnn_model.predict(first_word='sweet-talkin', song=songs[0], n_words=50)

"sweet-talkin i a.w.o.l lenya slam 'm shuttin she nobody i lido assassins and la they who all the finest shareef space bear anytime lebte sing oceans . 's like cop karma new gaining you particular offend constantly tired keep whip planes reign goes will al favourite found be-bop-a-lula low sighs away"